In [ ]:
#!/usr/bin/env python
# coding: utf-8

# # New Section

# # 1. 데이터 확인하기

# In[1]:


import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder

import matplotlib as mpl
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score
)
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

from sklearn.linear_model import LogisticRegression


# In[2]:


get_ipython().system('pip install xgboost')


# In[3]:


df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('submission_original (1).csv')


# In[4]:


df_train.head()


# In[5]:


df_test.head()


# ** 칼럼별 설명 **
# 1. bant_submit : MQL 구성 요소 중 예산, 고객의 직책/직급, 요구사항, 희망 납기일 4가지 항목에 대해 작성된 값의 비율
# 
# 2. customer_country: 고객의 국적
# 
# 3. business_unit : MQL 요청 상품에 대응하는 사업부
# 
# 4. com_reg_ver_win_rate :  Vertical Level 1, business unit, region을 기준으로 oppty 비율을 계산
# 
# 5. customer_idx : 고객의 회사명
# 
# 6. customer_type : 고객 유형
# 
# 7. enterprise : Global 기업인지, Small/Medium 규모의 기업인지
# 
# 8. historical_existing_cnt : 이전에 Converted(영업 전환) 되었던 횟수
# 
# 9. id_strategic_ver : (도메인 지식) 특정 사업부(Business Unit이 ID일 때), 특정 사업 영역(Vertical Level1)에 대해 가중치를 부여
# 
# 10. it_strategic_ver : (도메인 지식) 특정 사업부(Business Unit이 IT일 때), 특정 사업 영역(Vertical Level1)에 대해 가중치를 부여
# 
# 11. idit_strategic_ver : Id_strategic_ver이나 it_strategic_ver 값 중 하나라도 1의 값을 가지면 1 값으로 표현
# 
# 12. customer_job : 고객의 직업군
# 
# 13. lead_desc_length : 고객이 작성한 Lead Descriptoin 텍스트 총 길이
# 
# 14. inquiry_type : 고객의 문의 유형
# 
# 15. product_category : 요청 제품 카테고리
# 
# 16. product_subcategory	: 요청 제품 하위 카테고리
# 
# 17. product_modelname : 요청 제품 모델명
# 
# 18. customer_country.1 : 담당 자사 법인명 기반의 지역 정보(대륙)
# 
# 19. customer_position : 고객의 회사 직책
# 
# 20. response_corporate : 담당 자사 법인명
# 
# 21. expected_timeline : 고객의 요청한 처리 일정
# 
# 22. ver_cus : 특정 Vertical Level 1(사업영역) 이면서 Customer_type(고객 유형)이 소비자(End-user)인 경우에 대한 가중치
# 
# 23. ver_pro : 특정 Vertical Level 1(사업영역) 이면서 특정 Product Category(제품 유형)인 경우에 대한 가중치
# 
# 24. ver_win_rate_x : 전체 Lead 중에서 Vertical을 기준으로 Vertical 수 비율과 Vertical 별 Lead 수 대비 영업 전환 성공 비율 값을 곱한 값
# 
# 25. ver_win_ratio_per_bu : 특정 Vertical Level1의 Business Unit 별 샘플 수 대비 영업 전환된 샘플 수의 비율을 계산
# 
# 26. business_area : 고객의 사업 영역
# 
# 27. business_subarea : 고객의 세부 사업 영역
# 
# 28. lead_owner : 영업 담당자 이름
# 
# 29. is_converted : 영업 성공 여부

# # 2. 데이터 전처리

# 제거할 칼럼 추리기

# In[6]:


df_all = pd.read_csv('train.csv')


# In[7]:


df_all.info()


# In[8]:


df_test.info()


# In[9]:


df_all.plot(kind='scatter', x='com_reg_ver_win_rate', y='ver_win_ratio_per_bu')


# In[10]:


df_all.info()


# In[ ]:





# 각 칼럼별 결측치 비율 확인하고, 채우기

# In[11]:


df_all_na = (df_all.isnull().sum() / len(df_all)) * 100
df_all_na = df_all_na.drop(df_all_na[df_all_na == 0].index).sort_values(ascending=False)

missing_data = pd.DataFrame({"Missing Ratio" : df_all_na})
missing_data.head(20)


# In[12]:


df_all['business_unit'].unique()


# # idit_strategic_ver : 결측치 0으로 채우기

# 

# In[13]:


df_all['idit_strategic_ver'].fillna(0, inplace=True)


# In[14]:


df_all.info()


# 

# In[15]:


df_all['idit_strategic_ver'].unique()


# 
# 
# ```
# ## 코드로 형식 지정됨
# ```
# 
# # ver_win_ratio_per_bu

# In[16]:


df_all['ver_win_ratio_per_bu'].unique()
# 결측치를, 0이 아닌 중간값이나 평균값으로 채워 보는 건 어떨까?


# In[17]:


df_all['ver_win_ratio_per_bu'].describe()


# In[18]:


q1 = df_all['ver_win_ratio_per_bu'].quantile(0.25)
q3 = df_all['ver_win_ratio_per_bu'].quantile(0.75)
iqr = q3 - q1
q1


# In[19]:


stranger =df_all[(df_all['ver_win_ratio_per_bu'] > q3 + 1.5*iqr)]
len(stranger)


# In[20]:


df_all.info()


# In[21]:


#df_all[(df_all['customer_country'] == '//Mexico') & (df_all['enterprise'] == 'SMB')]


# # business_area, subarea 전처리

# In[22]:


df_all['business_area'].unique()


# In[23]:


len(df_all['business_area'].unique())


# In[24]:


df_all['business_area'].fillna('Unknown', inplace=True)


# In[25]:


len(df_all['business_area'].unique())


# In[26]:


# df_all['business'] = df_all['business_area'].map(str) + " " + df_all['business_subarea'].map(str)


# In[27]:


# len(df_all['business'].unique())


# In[28]:


df_all.head()


# # (잠시 보류) 어차피 안쓸거같지만 일단 놔두기

# # business_subarea 전처리

# In[29]:


len(df_all['business_subarea'].unique())


# In[30]:


df_all['business_subarea'].unique()


# In[31]:


df_all['business_subarea'].fillna('Unknown', inplace=True)


# In[32]:


df_all['business_subarea'].value_counts()


# # expected_timeline 전처리

# In[33]:


df_all['expected_timeline'].fillna('Unknown', inplace=True)


# In[34]:


len(df_all['expected_timeline'].unique())


# In[35]:


df_all['expected_timeline'].unique()


# In[36]:


df_all['expected_timeline'].value_counts()


# In[37]:


others = df_all['expected_timeline'].value_counts() < 5


# In[38]:


others_l = others[others]


# In[39]:


change = others_l.index


# In[40]:


for c in change:
    df_all.loc[df_all['expected_timeline'] == c, 'expected_timeline'] = "etc"


# In[41]:


df_all.loc[df_all['expected_timeline'] == 'less_than_3_months', 'expected_timeline'] = 'less than 3 months'
df_all.loc[df_all['expected_timeline'] == '3_months_~_6_months', 'expected_timeline'] = '3 months ~ 6 months'
df_all.loc[df_all['expected_timeline'] == '6_months_~_9_months', 'expected_timeline'] = '6 months ~ 9 months'
df_all.loc[df_all['expected_timeline'] == '9_months_~_1_year', 'expected_timeline'] = '9 months ~ 1 year'
df_all.loc[df_all['expected_timeline'] == 'being followed up.', 'expected_timeline'] = 'being followed up'


# In[42]:


df_all['expected_timeline'].value_counts()


# # inquiry_type 칼럼 전처리

# In[43]:


# inplace = True로 확실하게 기존 데이터 대체!!
df_all['inquiry_type'].fillna('None', inplace=True)


# In[44]:


df_all['inquiry_type'].unique()


# In[45]:


df_all['inquiry_type'].value_counts()


# In[46]:


len(df_all['inquiry_type'].unique())


# In[47]:


df_all['inquiry_type'].isnull()


# In[48]:


df_all.info()


# # customer_job 칼럼 전처리(이현주)
# 
# ---
# 
# 

# In[49]:


df_all['customer_job'].fillna('Inoccupied', inplace=True)


# In[50]:


len(df_all['customer_job'].unique())


# In[51]:


df_all['customer_job'].unique()


# In[52]:


df_all["job"]=df_all["customer_job"]


# In[53]:


engineering=['engineer','engineering', 'architect', 'project manager', 'designer', 'technical', 'systems engineer', 'construction manager']
information_technology=['research','Research','information technology', 'it specialist', 'it director', 'systems administrator', 'network administrator', 'it administrator', 'it manager', 'software developer']
business_and_management=['ceo/founder','general manager','programm-_und_projektmanagement ','executive','lead','buyer','general management','owner','manager','community and social services','real estate','support','Accounting','accounting','entrepreneurship', 'consulting', 'program and project management', 'sales', 'operations', 'administrative', 'business development', 'finance', 'marketing', 'quality assurance', 'legal', 'human resources']
healthcare=['healthcare services', 'radiology professional', 'medical imaging specialist', 'medical solution provider', 'pathologist', 'clinical specialist', 'doctor', 'surgery professional', 'healthcare professionals']
design_and_arts=['arquitecto/consultor','kreation_und_design','director','art and design','3d/vfx art',"film production",'designer','arts and design', 'graphic/color art', 'designer', 'curation', 'architect', 'design engineer', 'interior designer', 'digital project manager', 'product designer']
education=['education', 'teacher', 'educator', 'instructor', 'higher education (college & university)', 'k12 school']
retail_and_sales=['vertrieb','purchaser','product management',"purchasing",'sales', 'retailer/installer', 'sales executive', 'salesman', 'store promotions', 'sales operations', 'account management', 'field / outside sales', 'sales engineering', 'field marketing', 'retail', 'sales rep']
manufacturing=['manufacturing factory / plant', 'manufacturer', 'construction manager', 'engineering director', 'production manager', 'engineering & technical']
hospitality=['hotel manager', 'f&b director for bicycle casino', 'restaurant display', 'hotel tv', 'guestroom tv', 'hospitality']
media_and_communication=['media and communication', 'communication', 'media', 'marketing operations', 'marketing executive', 'advertising', 'broadcasting & media', 'media_e_comunicazione', 'medios_de_comunicación', 'média_és_kommunikáció', 'medien_und_kommunikation']
military=["military and protective services","Military and Protective Services","military and protective services"]
others= ['others','other',  'non specified', 'nothing', 'no requirment', 'n.a', 'otros', 'sonstiges', 'altro', 'egyéb', 'other stores']
inoccupied=["Inoccupied"]


# In[54]:


def find_category(value):
    if value in engineering:
        return "engineering"
    elif value in information_technology:
        return "information_technology"
    elif value in business_and_management:
        return "business_and_management"
    elif value in healthcare:
        return "healthcare"
    elif value in design_and_arts:
        return "design_and_arts"
    elif value in education:
        return "education"
    elif value in retail_and_sales:
        return "retail_and_sales"
    elif value in manufacturing:
        return "Manufacturing"
    elif value in hospitality:
        return "Hospitality"
    elif value in media_and_communication:
        return "media_and_communication"
    elif value in others:
        return "others"
    elif value in military:
        return "military"
    elif value in inoccupied:
        return "inoccupied"
    else:
        return "others"


# In[55]:


find_category("teacher")


# In[56]:


len(df_all["customer_job"])


# In[57]:


df_all["customer_job"].iloc[0]


# In[58]:


for i, row in df_all.iterrows():
    df_all.loc[i, "job"] = find_category(row["customer_job"])


# In[59]:


df_all['job'].unique()

df_all['customer_job'] = df_all['job']


# In[60]:


df_all.info()


# # customer_type
# 

# In[61]:


df_all['customer_type'].unique()


# In[62]:


df_all.loc[df_all['customer_type'] == 'Etc.', 'customer_type'] = 'Others'
df_all.loc[df_all['customer_type'] == 'Other', 'customer_type'] = 'Others'
df_all.loc[df_all['customer_type'] == 'Technical Assistant', 'customer_type'] = 'Others'
df_all.loc[df_all['customer_type'] == 'System Integrator', 'customer_type'] = 'Others'
df_all.loc[df_all['customer_type'] == 'Administrator', 'customer_type'] = 'Others'



df_all.loc[df_all['customer_type'] == 'Homeowner', 'customer_type'] = 'Home Owner'
df_all.loc[df_all['customer_type'] == 'End-Customer', 'customer_type'] = 'End Customer'
df_all.loc[df_all['customer_type'] == 'Specifier / Influencer', 'customer_type'] = 'Specifier/ Influencer'
df_all.loc[df_all['customer_type'] == 'Software / Solution Provider', 'customer_type'] = 'Software/Solution Provider'
df_all.loc[df_all['customer_type'] == 'Consultant', 'customer_type'] = 'Architect/Consultant'
df_all.loc[df_all['customer_type'] == 'Installer', 'customer_type'] = 'Installer/Contractor'
df_all.loc[df_all['customer_type'] == 'Distributor', 'customer_type'] = 'Dealer/Distributor'
df_all.loc[df_all['customer_type'] == 'Commercial end-user', 'customer_type'] = 'End-user'


df_all['customer_type'].fillna('Unknown', inplace=True)


# In[63]:


df_all['customer_type'].unique()


# #customer_country 칼럼 전처리하기(오래걸림 주의)

# In[64]:


len(df_all['customer_country'].unique())


# In[65]:


df_all.loc[df_all['customer_country'] == '//', 'customer_country'] = "Unknown"
df_all.loc[df_all['customer_country'] == '/  /', 'customer_country'] = "Unknown"
df_all.loc[df_all['customer_country'] == 'Corporate /  /', 'customer_country'] = "Unknown"


# In[66]:


df_all.loc[df_all['customer_country'].notnull() & df_all['customer_country'].str.contains('/India/'), 'customer_country'] = 'India'


# In[67]:


# 나라 이름만 추리기
country = []
for i in df_all['customer_country']:
    country.append(str(i).split('/')[-1])

print(country)

# 나라 이름만 남게 데이터 전처리
for j in range(len(df_all)):
    df_all.iloc[j, 1] = country[j]

 # email 항목 없에기
email = ['abdohabrouk@gmail.com', 'eslam.fiky@gmail.com',
       'mhaleem@proegypt.net', 'ihabsaad@hotmail.com',
       'Mohamed_mohamed@almansour.com.eg', 'Mostafasakr71@gmail.com',
       'keseba@swift-eg.com', 'mostafa@mark-all.com',
       'sameh_azema@yahoo.com', 'amrmelwani@gmail.com',
       'shswaify@hotmail.com', 'dr_mohhegab@yahoo.com', 'omeryousef235@yahoo.com', 'ahmedshanzlezeh0@gmail.com',
       'Jancel_moh@hotmail.com', 'youssefnagi345@gmail.com',
       'thawas@gmail.com', 'islam.zaky@yahoo.com', 'msamy6060@yahoo.com',
       'polessedeek@gmail.com', 'abdullahelkaramany831@gmail.com',
       'hafattah@yahoo.com', 'khaledelnour028@gmail.com',
       'sm@sanbouk.com.eg', 'emadorg@hotmail.com',
       'h_elgindi@hotmail.com', 'Infinity_elec@yahoo.com', 'bahaa.creative@gmail.com',
       'Mohi.ahmed.kassem@gmail.com', 'mahmoud.halawa19@gmail.com',
       'ahmed.elshwadfy@guarismo.com', 'eidoof45@gmail.com',
       'maleem@deltapower.net', 'mk601432@gmail.com', 'Infinity_elec@yahoo.com', 'bahaa.creative@gmail.com',
       'Mohi.ahmed.kassem@gmail.com', 'mahmoud.halawa19@gmail.com',
       'ahmed.elshwadfy@guarismo.com', 'eidoof45@gmail.com',
       'maleem@deltapower.net', 'mk601432@gmail.com', 'abdelaziz_eldesoky@hotmail.com',
       'samonline44@gmail.com', 'mahmodelrefaei@ymail.com',
       'walidwafeek@yahoo.com', 'waelahmed.1971@yahoo.com', 'helmy.3andy@gmail.com', 'Moh.enab9@gmail.com',
       'ahmedfathy868@gmail.com', 'sherif.hafez512@gmail.com',
       'dino_work2013@yahoo.com', 'ok.omar1985@gmail.com',
       'sarah.juman.1979@gmail.com', 'amrattia1981@yahoo.com',
       'Sheerbonnet@gmail.com', 'mrisuzu4@gmail.com',
       'ashraf.m.ezz@hotmail.com', 'ahmed_shemes22@yahoo.com', 'amrhussain@hotmail.com',
       'onsy.robeil@gmail.com', 'waelalshehawy@gmail.com',
       'taamere@gmail.com', 'Eslam_mahdi@hotmail.com',
       'adel.ouda85@gmail.com', 'ibrahimrd@gmail.com',
       'mostafa.h.hablas@gmail.com',
       'md_8821@yahoo.com']


for e in email:
    df_all.loc[df_all['customer_country'] == e, 'customer_country'] = "Unknown"


# In[68]:


# 미국 도시 데이터 하나로 전처리
us = (' Connecticut United States', 'Hacienda Heights, California, United States', ' US', ' OK 73601 United States', ' Clinton, OK 73601', '3801 E Willow St, Long Beach, CA 90815, EE. UU.',
       'Los Angeles, California, United States', '9110 Forest CrossingThe WoodlandsTX77381', 'Caldwell, New Jersey, United States United States',
       ' United States', '800 3rd Ave 3rd floor, New York, NY 10022', ' VT 05672', ' ND',
       'Zip 98433', ' Jamaica, NY 11417, USA',
       '723 S Valley Way, Palmer, AK 99645', '4278 S Buffalo St Orchard Park, NY 14127',
       '30 Cambria Ave, Pleasantville, NJ 08232', ' TX 77024',
       ' NJ 07013', ' KS 66217', 'US Virgin Islands', '13854 Lakeside Cir Sterling Heights, MI 48313',
       '65 Grove Street, Suite 204  Watertown, MA 02472', ' NY 11358',
       ' Arlington, NY 12603',
       '4278 S Buffalo St  Orchard Park, NY 14127', ' MO 63103', ' MO 63103, USA',
       '1156 W Armitage Ave suite b, Chicago, IL 60614, US.',
       ' Square at, 2662 Gateway Rd Suite 165, Carlsbad, CA 92009',
       '3100 Shore Drive Virginia Beach, VA 23451',
       '724 W Business US Highway 60, Dexter, MO 63841,',
       'Herndon, VA 20170', ' IL 61615', 'CA',
       '1817 S Washington AveTitusville, FL 32780, USA',
       '101 Clyde Ave Longwood, FL 32750, USA', '  FL 33404',
       ' 750 Florida Central Parkway Suite #100 Longwood, FL 32750',
       ' FL 33772', ' Wyckoff, NJ 07481, USA', 'GA 31405',
       'Jacksonville Florida', '12718 Kitten Trail, Hudson, FL 34669',
       '881 S Division St, Buffalo, NY 14210, United States',
       'West Caldwell NJ 07960 United States', 'New Hampshire', '63 Flushing Ave Bldg 303, Suite 804, Brooklyn, NY 11205, United States',
       '700 District Dr, Itasca, IL 60143 United States',
       'Little Rock, Arkansas, United States', ' CA 95618',
       '430 N Hamilton RdWhitehall, OH 43213, USA',
       '1775 Liberty Dr, Fort Belvoir, VA 22060, United States',
       '8569 Sudley Rd Ste C Historic District, Manassas, VA 20110, USA',
       ' MA 02780', ' CA 95814', '1385 N Weber Rd, Romeoville, IL 60446,',
       '1755 North Brown Rd. Suite 200 Lawrenceville, GA 30043',
       'Miami, Florida, United States',
       '3440 Rockefeller Ct Waldorf, MD 20602', 'FL 33716', ' FL 33442',
       'Nicolosi (CT), Italy',
       '990 Biscayne Dr, Concord, NC 28027, United States', ' UT 84116',
       '  CA 94555 United States',
       '463 Industrial Park Rd, Elysburg, PA 17824, US',
       '1491 Furnace St, Montgomery, AL 36104, United States', 'MN 55024',
       '135 Broadway, Schenectady, NY 12305, United States',
       '990 S Madera Ave, Kerman, CA 93630, United States', 'SD 57751',
       ' IL 60563United States', ' CA 92078', ' MO 64108.',
       ' IL 60638 United States',
       '7105 Northland Terrace N, Minneapolis, MN 55428',
       '101 MetLife Way, Cary, NC, 27513 – MET1',
       'Delmar, NY 12054 United States', ' IL 60191',  '600 5th Street', ' MA 01851', '305 John Street',
       '210 Route 4 East Fl 4', ' IL 60069',
       '1605 Ave. Ponce de León, Suite 400 San Juan, 00909, Puerto Rico',
       '2900 Highway 280Suite 250BirminghamAL35223',
       '100 Vestavia ParkwayBirminghamAL35216', '222 Maxine Dr',
       '275 Mishawum Road', '6601 Carroll Highlands Rd',
       '1380 Enterprise Dr', '3000 Montour Church Road',
       '9820HuntersvilleNC28078', '152 Bowdoin Street', '1001 Main St',
       '300 East Park Drive',
       '31700 Temecula Parkway, Temecula CA 92592 United States',
       '594 Howard StSan FranciscoCA94106',
       '594 Howard StSan FranciscoCA94105',
       '417 20th St NBirminghamAL35203', '24082 Carmel Dr',
       '106 Lakeview DrHomewoodAL35209',
       '1909 Forest Knoll DrHooverAL35244',  '1808 Lithgow RdCelinaTX75009', 'PO Box 112292CarrolltonTX75011',
       '5555', '7673 HEMPSTON CIR',
       '1503 LBJ ParkwaySuite 700Farmers BranchTX75234',
       '8454 Muirwood TrlFort WorthTX76137', '1560 Chance Ct',
       '2047  w summerdale ave', '955 Powell Ave SW',
       '825 Eastlake Ave E', '2266 Palmer Dr.',
       '5501 Headquarters DrPlanoTX75024', '15806 Longship Ct',
       '416 Panzano Drive', '17215 Welby Way',
       '6005 Commerce Dr. Ste. 300', '1715 Forest Cove Drive,  Apt 201',
       '1537 Rolling Hills Dr.', '126 Diablo Ranch Court',
       '3027 West Bay Villa Ave', '829 Jackson ave',
       '2217 HOUSTON DRIVEMelissaTX75454',
       '899 Kifer RoadSunnyvaleCA94086',
       '100 first stSan FranciscoCA94015',
       '6300 Harry Hines BlvdSte. 1400DallasTX75235',
       '100 Sabine River Dr.HuttoTX78634',
       '6111 W Plano Pkwy #2100PlanoTX75093',
       '1112 Badger Vine LaneArlingtonTX76005',
       '200 cabelLouisvilleKY40206', '9716 McFarring DrFort WorthTX76244',
       '1999 S Bascomb AveSuite 1000CampbellCA95008',
       '2350 Airport FrwyBedfordTX76022', '410 Baylor StAustinTX78703',
       '9111 cypress waters blvdDallasTX75038',
       '9420 West Sam Houston Pkwy NHoustonTX77018',
       '6564 Headquarters DrPlanoTX75051',
       '1209 Derby Run Carrollton, TX 75007CarrolltonTX75007',
       '1 Skyview drFort WorthTX76155', '11330 Clay RdHoustonTX77041',
       '602 Saber Creek Drive', '7700 West Sunrise BlvdPlantationFL33322',
       '230 Highland Ave, Suite 531SomervilleMA2143',
       '5301 Stevens Creek BlvdSanta ClaraCA95051',
       '252 Beech AvenueMelroseMA2176',
       'One Constitution RoadBostonMA2129',
       '9 Landsdowne StreetBostonMA2215', '6 Kimball LaneLynnfieldMA1940',
       '160 Gould St Ste 300, Needham Heights, Massachusetts 02494NeedhamMA2721',
       '77 Massachusetts AveCambridgeMA2139', '36 Margin StPeabodyMA1960',
       '700 Patroon Creek BlvdAlbanyNY12206',
       '400 Centre StreetNewtonMA2458', '400 Centre StNewtonMA2458',
       '2475 Washington BlvdOgdenUT84401',
       '9800 S. Monroe StreetSandyUT84070',
       '9820 Northcross Center CtHuntersvilleNC28078',
       '1100 ITBProvoUT84602', '45 n 200 wWillardUT84340',
       '21903 Ranier Ln', '545 West 111th st Suite 7C', '1 Hoag Dr.',
       '1036 n nalder st',
       '117 Bernal Rd Ste70-422San JoseSan JoseCA95119', '30 Winter Street Boston, MA 02108',  '2100 Engineer Rd, Oakland, CA 94607, United States',
       '239 Court St, Brooklyn, NY 11201, United States',
       ' 1110 Morse Rd, Columbus, Ohio, 43229, United States',
       ' CA 92618 United States', 'Jeffersonville, IN United States', ' United States 32901',
       'Four Embarcadero Center Suite 1400 #85, San Francisco, CA 94111 United States', 'United States 14503.',
       '335 League St S, Sulphur Springs, TX 75482',
       '2529 W Busch Blvd Suite 1000, Tampa, FL 33618, United States', '136 S IndustrialSalineMI48176', 'Nevada', ' St Cloud, MN 56303',
       '7703 Floyd Curl Dr MC 7800, San Antonio TX 78229 United States',
       '609 Medical Center Dr. Decatur, Texas, 76234',
       '500 N Highway 89 North, Prescott AZ 86313 United States',
       '8 Gay St, Christiana, PA 17509 United States', '275 John Hancock Rd. Taunton, MA, 02780', 'FL 33025', ' Ohio United States', ' MA 02062 United States', ' TX 78572 United States',
       'New York City Metropolitan Area, United States',
       '3804 W Broadway St, Ardmore, OK 73401, United States',
       '993 Niagara Ave, San Diego, CA 92107 United States',
       '6398 College Blvd, Overland Park, KS 66211, United States',
       'Grapevine, Texas, United States United States',
       '28001 238th St, Le Claire, IA 52753, USA', '14700 Caribbean Way', 'San Francisco, CA 94128, United States',
       'Ironhorse Customs LLC 4443 Genella Way North Las Vegas, NV 89031', '116 Village Blvd Suite 200, Princeton, NJ 08540 United States',
       '7610 Market St, Canton, MI 48187, United States',  'Bus Shed, 6501 Red Hook Rd #201, Nazareth, St Thomas 00802, U.S. Virgin Islands',
       ' OR 97128',
       '1600 Rosecrans Ave bldg 7 ste 101, Manhattan Beach, CA 90266,',
       ' CA 92657 United States', ' Warren, OH 44483.', ' IN 46601, EE. UU.', ', MO 64802',
       '6252 E Grant Rd suite 150 Tucson, AZ 85712', 'SC 29555',
       ' Benbrook, TX 76126', '9631 Liberty Rd B, Randallstown, MD 21133',
       '1800 Congress Ave., Austin, TX 78701', 'NY',
       ' VA 22209', ' NV 89119',
       '85 W Main St Suite C, Canton, GA 30114,',
       '1100 Lee Ave, Lafayette, LA 70501,',
       '1275 Sister Grove Rd Van Alstyne, TX 75495', ' CA 91915-6002',
       'Ohio', ' FL 32703', 'FL 33013',
       '13351 Dovers St, Waverly, NE 68462, United States',
       '717 General Booth BlvdVirginia Beach, VA 23451, USA', 'GA 30039',
       'Largo, Florida, United States United States',
       'P.O. Box 291992, Port Orange, FL 32129',
       '7581 S Franklin Way, Centennial, CO 80122, United States',
       '1222 Eastwick CirMurphy, TX 75094, United States', ' MO 64506',
       ' TX 75098', ' CO 80127',
       '2877 Prospect Rd, Fort Lauderdale, FL 33309', '823 Gateway Center Way, San Diego, CA 92102',
       'NY 10566 United States', 'MI 48827', 'TN 38120',
       '810 N KINGSTON DR PEORIA, IL 61604-2145', ' OH 45215, USA', '3 Nasson Avenue', '1397 Etowah DriveAtlantaGA30319',
       '450 Riverchase PkwyBirminghamAL35186',
       '3131 BriarPark Dr Suite 200HoustonTX77042',
       '4880 Stevens Creek BlvdSan JoseCA95129',
       '2367 N 2650 WFarr WestUT84404', 'Richardson, Texas, United States', '54 Sugar Creek Center Blvd #300, Sugar Land, TX 77478, United States',
       '1919 Minnesota Ct, Mississauga, ON L5N', ' UNITED STATES', ' NE', '5003 Lady Of The Lake Dr  Raleigh NC', ' Englewood, CO', ' FL',
       ' TX', ' CA', 'W126 N7449 Flint Drive Menomonee Falls VA',
       '233 South Beaudry Avenue  Los Angeles CA', ' NJ', '3 Center Plz Suite 330 Boston, MA 02108', ' NM', '603 Heritage Drive  Mount Juliet')


# In[69]:


for c in us:
    df_all.loc[df_all['customer_country'] == c, 'customer_country'] = "United States"

india = [' mumbai', 'Gujarat', ' Chennai', ' Telangana', ' bhilwara', ' bangalore', ' hyderabad',
       ' Anand Vihar Delhi', ' uttar pradesh', ' kerela', ' indore', ' India', ' gujarat', 'lucknow', 'mumbai', ' Pune', ' gurgaon']

for i in india:
    df_all.loc[df_all['customer_country'] == i, 'customer_country'] = "India"

unknown = ['nan', 'A', 'country', '48201']

for u in unknown:
    df_all.loc[df_all['customer_country'] == u, 'customer_country'] = 'Unknown'

df_all.loc[df_all['customer_country'] == 'Θέση Πέτσα Βακαλοπούλου ΒΙΟΠΑ Παλλήνης  153 51', 'customer_country'] = 'Greece'


# In[70]:


brazil = [' odisha', 'São Paulo, Pinheiros', 'Recife', 'Foz de Iguaçu - PR Avenida Tancredo Neves 6731 Jardim Itaipu',
          'Dourados', 'Cuiabá', 'Sao Paulo', 'Centro de Produção Audiovisual - Sesc São Paulo', 'João Pessoa', 'São Paulo',
          'Capão da canoa', 'Aparecida', 'Manaus', ' Brazil', 'Rio de Janeirol', ' RJ', 'Belo Horizonte', ' BR', 'Rio de Janeiro']
colombia = ['Cra. 51 #12 sur-75, San Fernando, Itagüi, Medellín, Guayabal, Medellín, Antioquia, Colombia',
            'Colombia C2: Solo información', 'Bucaramanga', 'Cartagena', 'Colombia - cartagena', 'CARRERA 11A 94-46 EDIFICIO CHICO 3000 PISO 3 BOGOTA',
            'COLOMBIA', 'Colombia  Cartagena', 'Barranquilla']
italy = ["Via dell'informatica 10 - 37036 San Martino Buon Albergo (Veneto), Italy", 'Via E. De Amicis, 23 . 90044 Carini (PA)', 'Firenze, Italy', 'Bari, Italy']


# In[71]:


for b in brazil:
    df_all.loc[df_all['customer_country'] == b, 'customer_country'] = "Brazil"

for c in colombia:
    df_all.loc[df_all['customer_country'] == c, 'customer_country'] = "Colombia"

for i in italy:
    df_all.loc[df_all['customer_country'] == i, 'customer_country'] = "Italy"


# In[72]:


fix = [' Philippines', ' Kenya', ' Indonesia',
       ' Thailand', ' South Africa', ' Botswana', ' United Kingdom',
       ' Ireland', ' Germany', ' Denmark', ' Nigeria', ' Canada', ' Serbia', ' Saudi Arabia',
       ' Singapore', ' Switzerland', ' Chile', ' Poland',
       ' Colombia', ' Algeria', ' Vietnam', ' Mexico', ' Panama',
       ' Honduras', ' Togo', ' Jordan', ' Israel', ' Romania', ' Belgium', ' Australia', ' Hong Kong', ' Bosnia and Herzegovina',
       ' Netherlands', ' Hungary', ' Maldives', ' Ecuador', ' Venezuela',
       ' Austria', ' Guatemala', ' Tunisia', ' Nicaragua', ' Somalia',
       ' China', ' Democratic Republic of the Congo',
       ' Dominican Republic', ' Greece', ' Qatar', ' Spain',
       ' Trinidad and Tobago', ' Kosovo', ' Bulgaria', ' Cambodia',
       ' Georgia', ' El Salvador', ' Jamaica', ' Cyprus', ' Bangladesh',
       ' Kuwait', ' Turks and Caicos Islands', ' Portugal', ' Uruguay',
       ' Costa Rica', ' Puerto Rico', ' St Maarten', ' Liberia',
       ' Burkina Faso', ' Nepal', ' Estonia', ' Afghanistan',
       ' Central African Republic', ' Bahrain', ' Paraguay', ' Malta',
       ' Bolivia', ' Iraq', ' Pakistan', ' France',
       ' Italy', ' Peru', ' Oman', ' U.A.E', ' Argentina', ' Türkiye']


# In[73]:


for f in fix:
    df_all.loc[df_all['customer_country'] == f, 'customer_country'] = f[1:]

egypt = [' Egypt', 'EGYPT']

for e in egypt:
    df_all.loc[df_all['customer_country'] == e, 'customer_country'] = "Egypt"

spain = ['CACERES', 'VALENCIA','GRAN CANARIAS PLAYA DEL INGLES', 'AGOST, ALICANTE', 'ELCHE, ALICANTE', 'MADRID', 'SPAIN']

for s in spain:
    df_all.loc[df_all['customer_country'] == s, 'customer_country'] = "Spain"

df_all.loc[df_all['customer_country'] == "Cote d'Ivoire", 'customer_country'] = 'Ivory Coast'
df_all.loc[df_all['customer_country'] == 'PerU', 'customer_country'] = 'Peru'
df_all.loc[df_all['customer_country'] == 'Ha Noi', 'customer_country'] = 'Vietnam'
df_all.loc[df_all['customer_country'] == 'UAE Dubai', 'customer_country'] = 'U.A.E'
df_all.loc[df_all['customer_country'] == 'ARGENTINA', 'customer_country'] = 'Argentina'
df_all.loc[df_all['customer_country'] == 'Türkiye', 'customer_country'] = 'Turkey'


# In[74]:


len(df_all['customer_country'].unique())


# In[75]:


# 이상하게 빈 칼럼들도 채우기
df_all.iloc[1799, 1] = 'Lewinion'
df_all.iloc[2749, 1] = 'Switzerland'
df_all.iloc[3085, 1] = 'Norway'
df_all.iloc[33089, 1] = 'Aruba'
df_all.iloc[3157, 1] = 'South Africa'
df_all.iloc[3308, 1] = 'Guam'
df_all.iloc[11421, 1] = 'Japan'
df_all.iloc[32435, 1] = 'Egypt'
df_all.iloc[34731, 1] = 'Fiji'
df_all.iloc[34795, 1] = 'Georgia'
df_all.iloc[20963, 1] = 'Sri Lanka'
df_all.iloc[20965, 1] = 'Albania'
df_all.iloc[5191, 1] = 'Bahama'
df_all.iloc[34693, 1] = 'Ivory Coast'
df_all.iloc[32503, 1] = 'Netherlands'
df_all.iloc[11346, 1] = 'Taiwan'
df_all.iloc[5594, 1] = 'Saipan'
df_all.iloc[34796, 1] = 'Hungary'
df_all.iloc[32604, 1] = 'Uganda'
df_all.iloc[32326, 1] = 'Suriname'
df_all.iloc[21888, 1] = 'Austria'
df_all.iloc[22432, 1] = 'Tanzania'
df_all.iloc[25353, 1] = 'Indonesia'
df_all.iloc[20413, 1] = 'Lesotho'
df_all.iloc[11328, 1] = 'Uzbekistan'
df_all.iloc[11538, 1] = 'Thailand'
df_all.iloc[34896, 1] = 'Jordan'
df_all.iloc[11549, 1] = 'Iceland'
df_all.iloc[11644, 1] = 'Monaco'
df_all.iloc[11592, 1] = 'Czech'


# In[76]:


singapore = [3096, 33078]
dominica = [3117, 32467]
poland = [3313, 11415, 59061, 59067]
puerto_rico = [3352, 32408, 33225, 33690]
china = [34716, 34902]
qatar = [34729, 34849]
kuw = [3604, 3605, 11312, 11394, 11449]
hkg = [11414, 34639]
jam = [5166, 11382]
mongo = [5168, 11541]
germ = [5188, 20585, 32508]
unknown = [2995, 5240, 12467, 12468, 31805, 56126]
canada = [5272, 11345, 11353, 11645, 12479, 32461, 32496, 32609, 34688, 34771]
spain = [5608, 34796]
uk = [10840, 12966]
phil = [14575, 14581, 14721, 14724, 14725, 14726, 14727, 14755,
        14785, 14788, 14911, 14927, 14953, 15139, 15140, 15141, 15169, 15193,
        15194, 15230, 15266, 15662, 15663, 15692, 15693, 21108, 32304, 32337,
        32463, 32542, 32544, 32655, 33110, 33226, 33227, 33253, 33696, 33706,
        33707, 33709, 33712, 33719, 33722, 33726, 33732, 33733, 33735, 33742,
        33748, 33753, 33756, 33760, 37067, 37134, 51464]
lux = [11316, 11461]
viet = [11483, 52024]
saudi = [11577, 11594, 34866, 51980]
malay = [12793, 33090, 34669]
australia = [33221, 34736]


# In[77]:


for s in singapore:
    df_all.iloc[s, 1] = 'Singapore'

for d in dominica:
    df_all.iloc[s, 1] = 'Dominican Republic'

for p in poland:
    df_all.iloc[p, 1] = 'Poland'

for p in puerto_rico:
    df_all.iloc[p, 1] = 'Puerto Rico'

for c in china:
    df_all.iloc[c, 1] = 'China'

for q in qatar:
    df_all.iloc[q, 1] = 'Qatar'

for k in kuw:
    df_all.iloc[k, 1] = 'Kuwait'

for h in hkg:
    df_all.iloc[h, 1] = 'Hong Kong'

for j in jam:
    df_all.iloc[j, 1] = 'Jamaica'

for m in mongo:
    df_all.iloc[m, 1] = 'Mongolia'

for g in germ:
    df_all.iloc[g, 1] = 'Germany'

for u in unknown:
    df_all.iloc[u, 1] = 'Unknown'

for c in canada:
    df_all.iloc[c, 1] = 'Canada'

for s in spain:
    df_all.iloc[s, 1] = 'Spain'

for p in phil:
    df_all.iloc[p, 1] = 'Philippines'

for l in lux:
    df_all.iloc[l, 1] = 'Luxembourg'

for v in viet:
    df_all.iloc[v, 1] = 'Vietnam'

for s in saudi:
    df_all.iloc[s, 1] = 'Saudi Arabia'

for m in malay:
    df_all.iloc[m, 1] = 'Malaysia'

df_all.loc[df_all['customer_country'] == 'Turks and Caicos Island', 'customer_country'] = 'Turks and Caicos Islands'

df_all.loc[df_all['customer_country'] == '', 'customer_country'] = 'United States'

df_all['customer_country'].value_counts() / len(df_all)


# In[78]:


df_all['customer_country'].unique()


# In[79]:


df_all.info()


# # Customer_position 전처리 (권민석)

# In[80]:


df_all['customer_position'].unique()


# In[81]:


len(df_all['customer_position'].unique())


# In[82]:


# 대소문자 통일
df_all['customer_position'] = df_all['customer_position'].str.lower()


# 유사한 항목 합치기
df_all['customer_position'].replace({
    'ceo/fundador': 'ceo/founder',
    'vicepresident': 'vice president',
    'entrylevel': 'entry level',
    'c-levelexecutive': 'c-level executive'
}, inplace=True)

# 결측치 처리
df_all['customer_position'].fillna('none', inplace=True)


# 전처리 후 고유한 값 확인
unique_values = df_all['customer_position'].unique()
print(unique_values)
len(df_all['customer_position'].unique())


# In[83]:


# 각 직책을 범주로 그룹화하기
customer_position_categories = {
    'entry level': 'Entry Level',
    'ceo/founder': 'Executive',
    'partner': 'Executive',
    'manager': 'Managerial',
    'vice president': 'Executive',
    'associate/analyst': 'Analyst/Consultant',
    'c-level executive': 'Executive',
    'director': 'Director',
    'intern': 'Intern/Trainee',
    'trainee': 'Intern/Trainee',
    'teacher': 'Education',
    'professor': 'Education',
    'assistant professor': 'Education',
    'lecturer': 'Education',
    'none': 'Unspecified/Other',
    'other': 'Unspecified/Other',
    'others': 'Unspecified/Other',
    'this is a consume display requirement for home purpose.': 'Unspecified/Other',
    'unpaid': 'Unspecified/Other',
    'av management': 'Technical',
    'founder': 'Executive',
    'engineering': 'Technical',
    'installer': 'Technical',
    'homeowner': 'Unspecified/Other',
    'consultant': 'Analyst/Consultant',
    'commercial end-user': 'Business',
    'employee': 'Unspecified/Other',
    'administrative': 'Administrative',
    'entrepreneurship': 'Business',
    'decision-influencer': 'Business',
    'decision maker': 'Business',
    'customer': 'Business',
    'not applicable': 'Unspecified/Other',
    'decision-maker': 'Business',
    'no influence': 'Unspecified/Other',
    'commercial consultant': 'Analyst/Consultant',
    'science teacher': 'Education',
    'decision influencer': 'Business',
    'architecture/consult': 'Technical',
    'architect/consultant': 'Technical',
    'exhibition': 'Event Management',
    'hospital': 'Medical',
    'end-user': 'Business',
    'government': 'Government/Public Sector',
    'manufacturer': 'Manufacturing',
    'software/solution provider': 'IT/Software',
    'system integrator': 'IT/Software',
    'medical device manufacturer': 'Medical',
    'distributor': 'Sales',
    'business unit director': 'Director',
    'business development': 'Business',
    'operations': 'Operations',
    'vp': 'Executive',
    'cargo': 'Logistics',
    'guest faculty': 'Education',
    'physics faculty': 'Education',
    'career coach': 'Education',
    'other - please specify - cedia association': 'Unspecified/Other',
    'leadership/executive office/owner': 'Executive',
    'product management': 'Management',
    'market intelligence/research': 'Research',
    'teacher/middle school coordinator': 'Education',
    'prof.': 'Education',
    'academic specialist': 'Education',
    'principal at oxford integrated pu science college': 'Education',
    'math and physics teacher': 'Education',
    'professor of mathematics': 'Education',
    'physics and mathematics teacher': 'Education',
    'assistant professor of english': 'Education',
    'educator': 'Education',
    'quantitative aptitude faculty': 'Education',
    'english trainer for ielts, toefl, pte, gre, sat exams.': 'Education',
    'associate professor': 'Education',
    'principal & director': 'Director',
    'business partner': 'Business',
    'hon dean': 'Education',
    'chairman': 'Executive',
    'pgt physics': 'Education',
    'education professional': 'Education',
    'chemistry teacher': 'Education',
    'director cum faculty at gaining apex coaching centre': 'Education',
    'academic coordinator/postgraduate teacher (accountancy, business studies)/tgt (ict)': 'Education',
    'senior lecturer': 'Education',
    'neet/olympiad expert faculty': 'Education',
    'assistant professor in electronics engg': 'Education',
    'professional trainer': 'Education',
    'co-founder': 'Executive',
    'chief executive officer': 'Executive',
    'subsidiary sales (ise)': 'Sales',
    'gerente': 'Executive',
    'medical imaging specialist': 'Medical',
    'tierarzt': 'Medical',
    'education': 'Education',
    'president': 'Executive',
    'business development/sales': 'Business',
    'sales': 'Sales',
    'técnico': 'Technical',
    'exhibitiontv': 'Event Management',
    'proprietário(a)': 'Business',
    'genel müdür': 'Executive',
    'mindenes': 'Unspecified/Other',
    'főorvos': 'Medical',
    'surgery professional': 'Medical',
    'pathologist': 'Medical',
    'research': 'Research',
    'the big boss': 'Executive',
    'radiology professional': 'Medical',
    'none': 'Unspecified/Other',
    'others': 'Unspecified/Other',
    'not applicable': 'Unspecified/Other',
    'this is a consume display requirement for home purpose.': 'Unspecified/Other'
}


# 범주에 라벨 할당하기
label_dict = {category: label for label, category in enumerate(set(customer_position_categories.values()))}

# 데이터프레임 수정
df_all['customer_position_category'] = df_all['customer_position'].map(customer_position_categories)
df_all['customer_position'] = df_all['customer_position_category'].map(label_dict)
df_all = df_all.drop('customer_position_category', axis=1)

# 결과 확인
print(df_all['customer_position'].head(100))
print(df_all['customer_position'].unique())
len(df_all['customer_position'].unique())
print(np.unique(df_all['customer_position'].values))


# In[84]:


df_all.info()


# # Product Category 전처리
# 

# In[85]:


df_all['product_category'].unique()


# In[86]:


df_all.loc[df_all['product_category'] == 'cac', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'sac', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'single package', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'cooler', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'ventilation', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'rac single cac', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'Air conditioning', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'aquecimento', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'scroll compressor', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'pendingin', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'ar condicionado residencial', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'ahu', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'lg home bliss air solution', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'lg paradise air solution', 'product_category'] = 'Air Conditioning'

df_all.loc[df_all['product_category'] == 'DRV variable refrigerant flow systems', 'product_category'] = 'Air Conditioning'
df_all.loc[df_all['product_category'] == 'lg salang air solution for dream homes', 'product_category'] = 'Air Conditioning'


# In[87]:


df_all.loc[df_all['product_category'] == '49vl5gm', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'uh', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '55vm5ea', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '55vm5jh', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '49vl5f', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'corpouh5f', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'bu50nst', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '86uh5f', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '55tc3d', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '43us660h na', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'corpuh5f', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'retaildigital', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'retaildigital', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'retaildigital', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'retaildigital', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'retaildigital', 'product_category'] = 'Displays'

df_all.loc[df_all['product_category'] == '28mq780', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'laec015', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'laec15', 'product_category'] = 'Displays'

df_all.loc[df_all['product_category'] == '55us660h0sdbwz', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'leadallin', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'laec015gnawz', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'gscd046', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '32lq621cbsbawz', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '43uh5fhawzm', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '49vl5gmawzm', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '55uq801c0sbbwz', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '43uq751c0sfbwz', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '43uq751c0sbbwz', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '49uh  49xf', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '50us660h0sdbwz', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '43uq751c0sfbwz', 'product_category'] = 'Displays'

df_all.loc[df_all['product_category'] == 'ur640', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'ur640s', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '43us660h0sdawz', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '50uq801c0sbbwz', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '32 pol', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '43 pol', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'lsca039', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'a definir', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'gscd100', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'gsca046', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '98uh5e', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'tr3', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == '55svh7fa', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'high inch 86  98 or 110', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'videwall', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'high brightness', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'lg magnit', 'product_category'] = 'Displays'
df_all.loc[df_all['product_category'] == 'ultra stretch series', 'product_category'] = 'Displays'



# In[88]:


df_all.loc[df_all['product_category'] == 'unitario', 'product_category'] = 'Home Appliances'

df_all.loc[df_all['product_category'] == 'pcwashing machine', 'product_category'] = 'Home Appliances'
df_all.loc[df_all['product_category'] == 'technical support', 'product_category'] = 'IT/Robot'
df_all.loc[df_all['product_category'] == 'allinone_rmk', 'product_category'] = 'IT/Robot'


# 

# In[89]:


pip install googletrans==4.0.0rc1


# In[90]:


pip install --upgrade pip


# In[91]:


# product_categories =df_all['transformed_category']

# 함수 임포트 및 정의
import re
from googletrans import Translator
import time

def translate_to_english(text):
    # Google Translate API를 사용하여 텍스트를 영어로 번역하는 함수
    translator = Translator()
    translator.raise_Exception = True
    translated_text = translator.translate(text, src='auto', dest='en')
    time.sleep(0.2)
    return translated_text.text

def categorize_product_category(product_categories):
    # 카테고리 키워드 설정
    categories_mapping = {
        'Air Conditioning': ['cac', 'air conditioner', 'split', 'vrf', 'chiller', 'heating', 'rac', ' ac', 'multi v', 'Conditioning', 'conditioning', 'heater', 'inverter'],
        'Displays': ['series','one quick', 'onequick', 'idb', 'signage', 'oled', 'led', 'tv', 'monitor', 'display', 'projector', 'video', 'aio'],
        'IT/Robot': ['monitor', 'pc', 'laptop', 'projector', 'cloud device', 'medical display', 'robots', 'software', 'webos'],
        'Energy': ['solar', 'energy storage system', 'ess'],
        'Home Appliances': ['washing machine', 'dryer', 'refrigerator', 'vacuum cleaner', 'microwave', 'builtin'],
        'Business': ['medical', 'surgical', 'education', 'hospitality', 'hotel', 'pro'],
        'Care': ['care', 'service']
    }

    categorized_product_categories = []

    for category in product_categories:
        # 모델명이나 기타 특수 문자 제거
        category = re.sub(r'[^\w\s]', '', category)

        # 영어가 아닌 경우 영어로 번역
        if not category.isascii():
            category = translate_to_english(category)

        # 카테고리에 맞게 분류
        matched_category = 'Others'
        for mapped_category, keywords in categories_mapping.items():
            if any(keyword in category.lower() for keyword in keywords):
                matched_category = mapped_category
                break

        categorized_product_categories.append(matched_category)

    return categorized_product_categories


# In[92]:


# 데이터프레임의 'product_train' 열을 문자열로 변환
df_all['product_category'] = df_all['product_category'].astype(str)

# 변환된 카테고리를 새로운 열에 추가
df_all['transformed_category'] = categorize_product_category(df_all['product_category'])

# 결과 출력
print(df_all)


# In[93]:


df_all['product_category'] = df_all['transformed_category']


df_all['product_category'].fillna('Unknown', inplace=True)


df_all = df_all.drop('transformed_category', axis =1 )


# In[94]:


df_all.info()


# # Inquiry Type (이주형)
# 

# In[95]:


get_ipython().system('pip install fuzzywuzzy python-Levenshtein')
get_ipython().system('pip install googletrans==4.0.0rc1')
get_ipython().system('pip install google-cloud-translate')


# In[96]:


import pandas as pd
from fuzzywuzzy import process, fuzz
from googletrans import Translator, LANGUAGES

#inq_type = df_all['inquiry_type']

inq_type = df_all['inquiry_type']


# In[97]:


num = inq_type.nunique()
print(f'# of Unique Features: {num}')


# In[98]:


missing_values = df_all['inquiry_type'].isnull().sum()
print("Missing Values:", missing_values)
df_all = df_all.fillna('empty')
missing_values = df_all['inquiry_type'].isnull().sum()
print("Missing Values:", missing_values)


# In[99]:


df_all['inquiry_type'] = df_all['inquiry_type'].str.lower()


# In[100]:


df_all['inquiry_type'].unique()


# In[101]:


df_all['inquiry_type'] = df_all['inquiry_type'].astype(str).str.replace('_', ' ')


# In[102]:


print(df_all['inquiry_type'].unique())


# In[103]:


def merge_similar_categories(types, threshold=50):
    unique_categories = types.dropna().unique()
    category_mapping = {}

    for category in unique_categories:
        if category not in category_mapping:
            matches = process.extract(category, unique_categories, scorer=fuzz.token_sort_ratio)
            similar_categories = [match[0] for match in matches if match[1] >= threshold]
            for similar_category in similar_categories:
                category_mapping[similar_category] = category

    merged_series = types.map(category_mapping).fillna(types)
    return merged_series, category_mapping


df_all['inquiry_type'] = df_all['inquiry_type'].astype(str)


df_all['inquiry_type'], category_mapping = merge_similar_categories(df_all['inquiry_type'])


# In[104]:


for original, merged in category_mapping.items():
    print(f'Original: {original} => Merged: {merged}')


# In[105]:


test = df_all['inquiry_type']
print(test.nunique())
print(test.unique())


# In[106]:


df_all.loc[df_all['inquiry_type'] == 'quotation or purchase consultation', 'inquiry_type'] = 'Quotation'
df_all.loc[df_all['inquiry_type'] == 'product information', 'inquiry_type'] = 'Product Information'
df_all.loc[df_all['inquiry_type'] == 'other', 'inquiry_type'] = 'Others'
df_all.loc[df_all['inquiry_type'] == 'technical support', 'inquiry_type'] = 'Inquiry'
df_all.loc[df_all['inquiry_type'] == 'trainings', 'inquiry_type'] = 'Others'
df_all.loc[df_all['inquiry_type'] == 'services', 'inquiry_type'] = 'Inquiry'
df_all.loc[df_all['inquiry_type'] == 'sales inquiry', 'inquiry_type'] = 'Inquiry'
df_all.loc[df_all['inquiry_type'] == 'request for partnership', 'inquiry_type'] = 'Quotation'
df_all.loc[df_all['inquiry_type'] == 'standalone', 'inquiry_type'] = 'Others'
df_all.loc[df_all['inquiry_type'] == '(select id needs)', 'inquiry_type'] = 'Others'
df_all.loc[df_all['inquiry_type'] == 'one quick:flex', 'inquiry_type'] = 'Others'
df_all.loc[df_all['inquiry_type'] == 'purchase', 'inquiry_type'] = 'Quotation'
df_all.loc[df_all['inquiry_type'] == 'customer suggestions', 'inquiry_type'] = 'Inquiry'
df_all.loc[df_all['inquiry_type'] == 'hospital tv', 'inquiry_type'] = 'Product Information'
df_all.loc[df_all['inquiry_type'] == 'i want to know the details about it', 'inquiry_type'] = 'Inquiry'
df_all.loc[df_all['inquiry_type'] == 'educational equipments', 'inquiry_type'] = 'Product Information'
df_all.loc[df_all['inquiry_type'] == 'digital platform', 'inquiry_type'] = 'Product Information'
df_all.loc[df_all['inquiry_type'] == 'tv interactive', 'inquiry_type'] = 'Product Information'
df_all.loc[df_all['inquiry_type'] == 'display textbook and photos', 'inquiry_type'] = 'Product Information'
df_all.loc[df_all['inquiry_type'] == 'high inch 86 / 98 or 110', 'inquiry_type'] = 'Product Information'
df_all.loc[df_all['inquiry_type'] == 'first info and pricing', 'inquiry_type'] = 'Quotation'
df_all.loc[df_all['inquiry_type'] == 'idb', 'inquiry_type'] = 'Others'
df_all.loc[df_all['inquiry_type'] == 'not specified', 'inquiry_type'] = 'Others'
df_all.loc[df_all['inquiry_type'] == 'for school', 'inquiry_type'] = 'Product Information'
df_all.loc[df_all['inquiry_type'] == 'video wall', 'inquiry_type'] = 'Product Information'


# In[107]:


print(df_all['inquiry_type'])


# In[108]:


df_all['inquiry_type'].nunique()
df_all['inquiry_type'].unique()


# In[109]:


df_all.info()


# # Data 열 제거
# 

# In[110]:


# 칼럼 드랍하기
# 드랍할 칼럼 추리기
drop = ['customer_idx', 
        'ver_win_rate_x', 'customer_country.1', 'business_subarea',
        'historical_existing_cnt', 'product_subcategory', 'product_modelname'
        , 'job', 'is_converted','id_strategic_ver','it_strategic_ver']

df_all = df_all.drop(df_all[drop], axis=1)



# In[111]:


df_all.info()


# # Train data 인코딩

# In[112]:


#베이스 라인 참고해서 만든 라벨 인코딩 (컬럼도 다 채워 넣었습니다)
save_dict = {}
def label_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""

    my_dict = {}

    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    for idx, value in enumerate(sorted(series.unique())):
        my_dict[value] = idx
    series = series.map(my_dict)
    save_dict.update(my_dict)
    return series
def label_test_encoding(series: pd.Series) -> pd.Series:
    """범주형 데이터를 시리즈 형태로 받아 숫자형 데이터로 변환합니다."""


    # 모든 요소를 문자열로 변환
    series = series.astype(str)

    series = series.map(save_dict)

    return series

# 레이블 인코딩할 칼럼들
label_columns = [
    "customer_country",
    "business_unit",
    "customer_type",
    "enterprise",
    "customer_job",
    "inquiry_type",
    "product_category",
    "customer_position",
    "response_corporate",
    "expected_timeline",
    "business_area",
    "bant_submit",
    "com_reg_ver_win_rate",
    "idit_strategic_ver",
    "lead_desc_length",    
    "ver_cus",    
    "ver_pro",
    "ver_win_ratio_per_bu",
    "lead_owner"
]

for col in label_columns:
    df_all[col] = label_encoding(df_all[col])


# # TEST 데이터셋 전처리 (위 코드 사용)

# In[113]:


df_test.info()


# In[114]:


df_test['idit_strategic_ver'].fillna(0, inplace=True)


# In[115]:


df_test['business_area'].fillna('Unknown', inplace=True)


# In[116]:


df_test['business_subarea'].fillna('Unknown', inplace=True)


# In[117]:


df_test['expected_timeline'].fillna('Unknown', inplace=True)


# In[118]:


test_others = df_test['expected_timeline'].value_counts() < 5


# In[119]:


test_others_l = test_others[test_others]


# In[120]:


change = test_others_l.index


# In[121]:


for c in change:
    df_test.loc[df_test['expected_timeline'] == c, 'expected_timeline'] = "etc"


# In[122]:


df_test.loc[df_test['expected_timeline'] == 'less_than_3_months', 'expected_timeline'] = 'less than 3 months'
df_test.loc[df_test['expected_timeline'] == '3_months_~_6_months', 'expected_timeline'] = '3 months ~ 6 months'
df_test.loc[df_test['expected_timeline'] == '6_months_~_9_months', 'expected_timeline'] = '6 months ~ 9 months'
df_test.loc[df_test['expected_timeline'] == '9_months_~_1_year', 'expected_timeline'] = '9 months ~ 1 year'
df_test.loc[df_test['expected_timeline'] == 'being followed up.', 'expected_timeline'] = 'being followed up'


# In[123]:


# inplace = True로 확실하게 기존 데이터 대체!!
df_test['inquiry_type'].fillna('None', inplace=True)


# In[124]:


df_test['customer_job'].fillna('Inoccupied', inplace=True)


# In[125]:


df_test["job"]=df_test["customer_job"]


# In[126]:


for i, row in df_test.iterrows():
    df_test.loc[i, "job"] = find_category(row["customer_job"])


# In[127]:


df_test['customer_job'] = df_test['job']


# In[128]:


df_test.loc[df_test['customer_type'] == 'Etc.', 'customer_type'] = 'Others'
df_test.loc[df_test['customer_type'] == 'Other', 'customer_type'] = 'Others'
df_test.loc[df_test['customer_type'] == 'Technical Assistant', 'customer_type'] = 'Others'
df_test.loc[df_test['customer_type'] == 'System Integrator', 'customer_type'] = 'Others'
df_test.loc[df_test['customer_type'] == 'Administrator', 'customer_type'] = 'Others'



df_test.loc[df_test['customer_type'] == 'Homeowner', 'customer_type'] = 'Home Owner'
df_test.loc[df_test['customer_type'] == 'End-Customer', 'customer_type'] = 'End Customer'
df_test.loc[df_test['customer_type'] == 'Specifier / Influencer', 'customer_type'] = 'Specifier/ Influencer'
df_test.loc[df_test['customer_type'] == 'Software / Solution Provider', 'customer_type'] = 'Software/Solution Provider'
df_test.loc[df_test['customer_type'] == 'Consultant', 'customer_type'] = 'Architect/Consultant'
df_test.loc[df_test['customer_type'] == 'Installer', 'customer_type'] = 'Installer/Contractor'
df_test.loc[df_test['customer_type'] == 'Distributor', 'customer_type'] = 'Dealer/Distributor'
df_test.loc[df_test['customer_type'] == 'Commercial end-user', 'customer_type'] = 'End-user'


df_test['customer_type'].fillna('Unknown', inplace=True)


# In[129]:


df_test.loc[df_test['customer_country'] == '/  /', 'customer_country'] = "Unknown"
df_test.loc[df_test['customer_country'] == 'Corporate /  /', 'customer_country'] = "Unknown"


# In[130]:


test_country = []
for i in df_test['customer_country']:
    test_country.append(str(i).split('/')[-1])


# 나라 이름만 남게 데이터 전처리
for j in range(len(df_test)):
    df_test.iloc[j, 2] = test_country[j]
for e in email:
    df_test.loc[df_test['customer_country'] == e, 'customer_country'] = "Unknown"


# In[131]:


for c in us:
    df_test.loc[df_test['customer_country'] == c, 'customer_country'] = "United States"

for i in india:
    df_test.loc[df_test['customer_country'] == i, 'customer_country'] = "India"

for u in unknown:
    df_test.loc[df_test['customer_country'] == u, 'customer_country'] = 'Unknown'

df_test.loc[df_test['customer_country'] == 'Θέση Πέτσα Βακαλοπούλου ΒΙΟΠΑ Παλλήνης  153 51', 'customer_country'] = 'Greece'


# In[132]:


df_test.loc[df_test['customer_country'] == "Cote d'Ivoire", 'customer_country'] = 'Ivory Coast'
df_test.loc[df_test['customer_country'] == 'PerU', 'customer_country'] = 'Peru'
df_test.loc[df_test['customer_country'] == 'Ha Noi', 'customer_country'] = 'Vietnam'
df_test.loc[df_test['customer_country'] == 'UAE Dubai', 'customer_country'] = 'U.A.E'
df_test.loc[df_test['customer_country'] == 'ARGENTINA', 'customer_country'] = 'Argentina'
df_test.loc[df_test['customer_country'] == 'Türkiye', 'customer_country'] = 'Turkey'


# In[133]:


df_test.loc[df_test['customer_country'] == 'Turks and Caicos Island', 'customer_country'] = 'Turks and Caicos Islands'

df_test.loc[df_test['customer_country'] == '', 'customer_country'] = 'United States'


# In[134]:


df_test['customer_country'] = df_test['customer_country'].str.lstrip()


# In[135]:


df_test['customer_country'].unique()


# In[136]:


# 대소문자 통일
df_test['customer_position'] = df_test['customer_position'].str.lower()


# 유사한 항목 합치기
df_test['customer_position'].replace({
    'ceo/fundador': 'ceo/founder',
    'vicepresident': 'vice president',
    'entrylevel': 'entry level',
    'c-levelexecutive': 'c-level executive'
}, inplace=True)

# 결측치 처리
df_test['customer_position'].fillna('none', inplace=True)


# 전처리 후 고유한 값 확인
test_unique_values = df_test['customer_position'].unique()


# In[137]:


df_test.info()


# In[138]:


# 데이터프레임 수정
df_test['customer_position_category'] = df_test['customer_position'].map(customer_position_categories)
df_test['customer_position'] = df_test['customer_position_category'].map(label_dict)
df_test = df_test.drop('customer_position_category', axis=1)


# In[139]:


df_test.loc[df_test['product_category'] == 'cac', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'sac', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'single package', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'cooler', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'ventilation', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'rac single cac', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'Air conditioning', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'aquecimento', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'scroll compressor', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'pendingin', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'ar condicionado residencial', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'ahu', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'lg home bliss air solution', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'lg paradise air solution', 'product_category'] = 'Air Conditioning'

df_test.loc[df_test['product_category'] == 'DRV variable refrigerant flow systems', 'product_category'] = 'Air Conditioning'
df_test.loc[df_test['product_category'] == 'lg salang air solution for dream homes', 'product_category'] = 'Air Conditioning'


# In[140]:


df_test.loc[df_test['product_category'] == '49vl5gm', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'uh', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '55vm5ea', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '55vm5jh', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '49vl5f', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'corpouh5f', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'bu50nst', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '86uh5f', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '55tc3d', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '43us660h na', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'corpuh5f', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'retaildigital', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'retaildigital', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'retaildigital', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'retaildigital', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'retaildigital', 'product_category'] = 'Displays'

df_test.loc[df_test['product_category'] == '28mq780', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'laec015', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'laec15', 'product_category'] = 'Displays'

df_test.loc[df_test['product_category'] == '55us660h0sdbwz', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'leadallin', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'laec015gnawz', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'gscd046', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '32lq621cbsbawz', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '43uh5fhawzm', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '49vl5gmawzm', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '55uq801c0sbbwz', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '43uq751c0sfbwz', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '43uq751c0sbbwz', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '49uh  49xf', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '50us660h0sdbwz', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '43uq751c0sfbwz', 'product_category'] = 'Displays'

df_test.loc[df_test['product_category'] == 'ur640', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'ur640s', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '43us660h0sdawz', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '50uq801c0sbbwz', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '32 pol', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '43 pol', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'lsca039', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'a definir', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'gscd100', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'gsca046', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '98uh5e', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'tr3', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == '55svh7fa', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'high inch 86  98 or 110', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'videwall', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'high brightness', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'lg magnit', 'product_category'] = 'Displays'
df_test.loc[df_test['product_category'] == 'ultra stretch series', 'product_category'] = 'Displays'


# In[141]:


df_test.loc[df_test['product_category'] == 'unitario', 'product_category'] = 'Home Appliances'

df_test.loc[df_test['product_category'] == 'pcwashing machine', 'product_category'] = 'Home Appliances'
df_test.loc[df_test['product_category'] == 'technical support', 'product_category'] = 'IT/Robot'
df_test.loc[df_test['product_category'] == 'allinone_rmk', 'product_category'] = 'IT/Robot'


# In[142]:


# 데이터프레임의 'product_train' 열을 문자열로 변환
df_test['product_category'] = df_test['product_category'].astype(str)

# 변환된 카테고리를 새로운 열에 추가
df_test['transformed_category'] = categorize_product_category(df_test['product_category'])


# In[143]:


df_test['product_category'] = df_test['transformed_category']


df_test['product_category'].fillna('Unknown', inplace=True)




df_test = df_test.drop('transformed_category', axis =1 )


# In[144]:


df_test['inquiry_type'] = df_test['inquiry_type'].astype(str).str.replace('_', ' ')
df_test['inquiry_type'] = df_test['inquiry_type'].astype(str)


df_test['inquiry_type'], category_mapping = merge_similar_categories(df_test['inquiry_type'])


# In[145]:


df_test.loc[df_test['inquiry_type'] == 'quotation or purchase consultation', 'inquiry_type'] = 'Quotation'
df_test.loc[df_test['inquiry_type'] == 'product information', 'inquiry_type'] = 'Product Information'
df_test.loc[df_test['inquiry_type'] == 'other', 'inquiry_type'] = 'Others'
df_test.loc[df_test['inquiry_type'] == 'technical support', 'inquiry_type'] = 'Inquiry'
df_test.loc[df_test['inquiry_type'] == 'trainings', 'inquiry_type'] = 'Others'
df_test.loc[df_test['inquiry_type'] == 'services', 'inquiry_type'] = 'Inquiry'
df_test.loc[df_test['inquiry_type'] == 'sales inquiry', 'inquiry_type'] = 'Inquiry'
df_test.loc[df_test['inquiry_type'] == 'request for partnership', 'inquiry_type'] = 'Quotation'
df_test.loc[df_test['inquiry_type'] == 'standalone', 'inquiry_type'] = 'Others'
df_test.loc[df_test['inquiry_type'] == '(select id needs)', 'inquiry_type'] = 'Others'
df_test.loc[df_test['inquiry_type'] == 'one quick:flex', 'inquiry_type'] = 'Others'
df_test.loc[df_test['inquiry_type'] == 'purchase', 'inquiry_type'] = 'Quotation'
df_test.loc[df_test['inquiry_type'] == 'customer suggestions', 'inquiry_type'] = 'Inquiry'
df_test.loc[df_test['inquiry_type'] == 'hospital tv', 'inquiry_type'] = 'Product Information'
df_test.loc[df_test['inquiry_type'] == 'i want to know the details about it', 'inquiry_type'] = 'Inquiry'
df_test.loc[df_test['inquiry_type'] == 'educational equipments', 'inquiry_type'] = 'Product Information'
df_test.loc[df_test['inquiry_type'] == 'digital platform', 'inquiry_type'] = 'Product Information'
df_test.loc[df_test['inquiry_type'] == 'tv interactive', 'inquiry_type'] = 'Product Information'
df_test.loc[df_test['inquiry_type'] == 'display textbook and photos', 'inquiry_type'] = 'Product Information'
df_test.loc[df_test['inquiry_type'] == 'high inch 86 / 98 or 110', 'inquiry_type'] = 'Product Information'
df_test.loc[df_test['inquiry_type'] == 'first info and pricing', 'inquiry_type'] = 'Quotation'
df_test.loc[df_test['inquiry_type'] == 'idb', 'inquiry_type'] = 'Others'
df_test.loc[df_test['inquiry_type'] == 'not specified', 'inquiry_type'] = 'Others'
df_test.loc[df_test['inquiry_type'] == 'for school', 'inquiry_type'] = 'Product Information'
df_test.loc[df_test['inquiry_type'] == 'video wall', 'inquiry_type'] = 'Product Information'


# In[146]:


df_test.info()


# # TEST 데이터 인코딩 및 열 삭제

# In[147]:


df_test = df_test.drop(df_test[drop], axis=1)


# In[148]:


for col in label_columns:
    df_test[col] = label_test_encoding(df_test[col])


# In[149]:


X_train=df_all
y=df_train['is_converted']
X_test = df_test


# # 모델 훈련

# In[150]:


x_train, x_val, y_train, y_val = train_test_split(
    X_train,y,
    test_size=0.11,
    shuffle=True,
    random_state=400,
)


# In[151]:


def get_clf_eval(y_test, y_pred=None):
    confusion = confusion_matrix(y_test, y_pred, labels=[True, False])
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, labels=[True, False])
    recall = recall_score(y_test, y_pred)
    F1 = f1_score(y_test, y_pred, labels=[True, False])

    print("오차행렬:\n", confusion)
    print("\n정확도: {:.4f}".format(accuracy))
    print("정밀도: {:.4f}".format(precision))
    print("재현율: {:.4f}".format(recall))
    print("F1: {:.4f}".format(F1))


# In[152]:


# from sklearn.svm import SVC
# from sklearn.metrics import accuracy_score
# from sklearn.metrics import f1_score

# svm_clf = SVC(random_state=24,C=1,kernel='poly')
# svm_clf.fit(x_train, y_train)
# svm_pred = svm_clf.predict(X_test)

# X_test['is_converted']= svm_pred

# #print("SVM Score :{0:.4f}".format(accuracy_score(y_val, svm_pred)))
# #print("F1 score of model : {0:.4f}".format(f1_score(y_val,svm_pred,average='macro')))


# In[153]:


pip install -U imbalanced-learn


# In[154]:


import imblearn
from imblearn.under_sampling import RandomUnderSampler

x_train, y_train = RandomUnderSampler(random_state=2021).fit_resample(x_train, y_train)


# In[155]:


from xgboost import XGBClassifier

xgbc = XGBClassifier()
xgbc.fit(x_train, y_train)

pred = xgbc.predict(x_val)
get_clf_eval(y_val, pred)


# In[156]:


X_test.info()


# In[157]:


df_sub = pd.read_csv('submission_original (1).csv')


# In[158]:


test_pred = xgbc.predict(X_test.drop("id",axis=1))


# In[159]:


test_pred.shape


# In[160]:


df_sub.shape


# In[161]:


df_sub['is_converted']=test_pred


# In[162]:


df_sub['is_converted'].value_counts()


# In[163]:


df_sub.to_csv('submission.csv')


# 
